# Computational assignment 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from matplotlib import rc
# Set common figure parameters:
newparams = {'axes.labelsize': 11, 'axes.linewidth': 1, 'savefig.dpi': 300, 
             'lines.linewidth': 1.0, 'figure.figsize': (20, 10),
             'ytick.labelsize': 10, 'xtick.labelsize': 10,
             'ytick.major.pad': 5, 'xtick.major.pad': 5,}
plt.rcParams.update(newparams)

In [ ]:
# Setup
m = 1 #kg
R = 1 #m
g = 9.81 #m/s^2

theta_0 = 0.2 # rad, inital angle
omega_0 = 0.0 # rad/s, inital angular velocity

end_time = 7 # s, the time at which the simulation will end

The problem consists of two first-order differential equations:
$$
\dot{\theta} = \omega\\
\dot{\omega} = \frac{F_{\perp}}{mR}.
$$

Let 
$$
y = 
\begin{bmatrix}
\theta\\
\omega
\end{bmatrix},
$$
then
$$
y' = 
\begin{bmatrix}
y_2\\
-\sin(y_1) / mR
\end{bmatrix}
= f(y).
$$

We will do the euler step:
$$
y(t+dt) = y(t) + f(y(t)) dt
$$

In [ ]:
def f(y):
    """f(y) = y'  """
    return np.array([y[1], -g*np.sin(y[0])/R])

In [ ]:
def euler_step(y, f, dt):
    """Returns an euler-approximation of y(t+dt) given y(t)"""
    return y + f(y)*dt

In [ ]:
def euler_solve(y, f, dt, num_iterations):
    """Euler's method"""
    for i in range(num_iterations - 1):
        y[i+1] = euler_step(y[i], f, dt)
    return y

def euler_cromer_solve(y, f, dt, num_iterations):
    """Euler-Cromer's method"""
    for i in range(num_iterations - 1):
        theta, omega = euler_step(y[i], f, dt)
        theta = y[i,0] + omega*dt
        y[i+1] = np.array([theta, omega])
    return y

def rk4_solve(y, f, dt, num_iterations):
    """Runge-Kutta 4"""
    for i in range(num_iterations - 1):
        k_1 = f(y[i])*dt
        k_2 = f(y[i] + k_1/2)*dt
        k_3 = f(y[i] + k_2/2)*dt
        k_4 = f(y[i] + k_3)*dt
        y[i+1] = y[i] + (k_1 + 2*k_2 + 2*k_3 + k_4)/6
    return y

In [ ]:
def kinetic_energy(omega):
    return 0.5*m*R**2*omega**2

def potential_energy(theta):
    return m*g*R*(1-np.cos(theta))

def total_energy(y1, y2):
    return kinetic_energy(y2) + potential_energy(y1)

def plt_method_energy(t, theta, omega, name):
    plt.plot(t, potential_energy(theta), label=f"{name} potential")
    plt.plot(t, kinetic_energy(theta), label=f"{name} kinetic")
    plt.plot(t, total_energy(theta, omega), label=f"{name} total")

## Part one, Euler method for different time-delta

In [ ]:
# Using Euler's method

def do_method(end_time, dt, solver=euler_solve, theta_0=theta_0, omega_0=omega_0):
    """Use method given end_time, dt
    Returns:
     t: array of times
     y: 2D-array with [theta, omega]"""
    num_iterations = int(np.ceil(end_time/dt))
    y = np.zeros((num_iterations, 2))
    y[0] = np.array([theta_0, omega_0])
    y = solver(y, f, dt, num_iterations)
    t = np.arange(0, end_time, dt)
    
    return t, y

def analytical(t, theta_0=theta_0, omega_0=omega_0):
    """Analytical solution to small-angle approx. (ie. harm. osc.)"""
    omega = np.sqrt(g/R) # Angular frequency
    A = np.sqrt(theta_0**2 + (omega_0/omega)**2)
    phi = np.arctan(-omega_0/(omega*theta_0))
    return A*np.sin(omega*t + phi + np.pi/2)

# Euler's method for different time deltas
dt_list = [1e-4, 1e-3, 5e-3, 1e-2, 5e-2]
y_euler_dt = {}
for dt in dt_list:
    y_euler_dt[dt] = do_method(end_time, dt)

for i in range(len(dt_list)):
    plt.subplot(len(dt_list), 2, 2*i+1)
    dt = dt_list[i]
    t = y_euler_dt[dt][0]
    y = y_euler_dt[dt][1]
    plt.title(f"dt = {dt}")
    plt.plot(t, y[:,0], label="Numerical")
    t_n = np.linspace(0, end_time, 100)
    plt.plot(t_n, analytical(t_n), label="Analytical")
    plt.legend()
    
    plt.subplot(len(dt_list), 2, 2*i+2)
    plt_method_energy(t, y[:,0], y[:,1], "Euler")
    plt.legend()
    
plt.tight_layout()

We begin to see a serious deviation from energy conservation at dt=5e-3

## Part two, more sophisticated methods

In [ ]:
dt = 0.05

y_dict = {}
method_list = [euler_solve, euler_cromer_solve, rk4_solve]
for method in method_list:
    t, y_dict[method.__repr__()] = do_method(end_time, dt, method)

## Let's do some plotting!!
First is $\omega(t)$ and energy for all methods

In [ ]:
plt.title("$\\theta(t)$")
for method in method_list:
    y = y_dict[method.__repr__()]
    theta = y[:,0]
    omega = y[:,1]
    plt.plot(t, theta, label=method.__doc__)
    plt.xlabel("t [s]")
    plt.ylabel("$\\theta(t)$ [rad]")
    plt.legend()
    plt.show()
    
    plt_method_energy(t, theta, omega, method.__doc__)
    plt.xlabel("t [s]")
    plt.ylabel("$E$ [J]")
    plt.legend()
    plt.show()

Plotted in phase space:

In [ ]:
for method in method_list:
    y = y_dict[method.__repr__()]
    theta = y[:,0]
    omega = y[:,1]
    plt.plot(theta, omega, label=method.__doc__)
    plt.xlabel("$\\theta$")
    plt.ylabel("$\omega$")
    plt.legend()
    plt.show()